In [1]:
data = open('kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 137624 chars, 80 unique


In [2]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'l': 0, '0': 1, 'X': 2, 'm': 3, 'u': 4, '2': 5, 'T': 6, '7': 7, '5': 8, '%': 9, '@': 10, '(': 11, '*': 12, 'S': 13, 'z': 14, '6': 15, 'J': 16, '9': 17, 'F': 18, 'B': 19, 'W': 20, '4': 21, 'r': 22, 'o': 23, 't': 24, 'q': 25, '!': 26, ':': 27, 'y': 28, '"': 29, 'L': 30, 'C': 31, 'O': 32, '.': 33, 'Y': 34, ',': 35, 'H': 36, 's': 37, '-': 38, 'x': 39, 'p': 40, 'P': 41, "'": 42, 'G': 43, 'E': 44, 'M': 45, 'U': 46, 'n': 47, 'e': 48, 'k': 49, 'v': 50, 'R': 51, 'h': 52, 'I': 53, 'a': 54, 'f': 55, 'c': 56, 'V': 57, '/': 58, '8': 59, '\n': 60, 'K': 61, 'w': 62, 'D': 63, ' ': 64, 'j': 65, 'N': 66, 'g': 67, 'A': 68, 'Q': 69, '$': 70, 'd': 71, ')': 72, 'b': 73, '3': 74, 'ç': 75, 'i': 76, ';': 77, '1': 78, '?': 79}
{0: 'l', 1: '0', 2: 'X', 3: 'm', 4: 'u', 5: '2', 6: 'T', 7: '7', 8: '5', 9: '%', 10: '@', 11: '(', 12: '*', 13: 'S', 14: 'z', 15: '6', 16: 'J', 17: '9', 18: 'F', 19: 'B', 20: 'W', 21: '4', 22: 'r', 23: 'o', 24: 't', 25: 'q', 26: '!', 27: ':', 28: 'y', 29: '"', 30: 'L', 31: 'C', 32: 'O', 

In [3]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]


In [4]:
#model parameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [5]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [6]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],50)

----
 c$(kxs3Xf$OJ$*f/vYUST@pUmbq9w!$x37e/;
WcfBkVlq?8OX 
----


In [7]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", targets)

inputs [32, 47, 48, 64, 3, 23, 22, 47, 76, 47, 67, 35, 64, 62, 52, 48, 47, 64, 43, 22, 48, 67, 23, 22, 64]
targets [47, 48, 64, 3, 23, 22, 47, 76, 47, 67, 35, 64, 62, 52, 48, 47, 64, 43, 22, 48, 67, 23, 22, 64, 13]


In [8]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0   

In [9]:
n, p = 0, 0
while n<=100*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 50)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 109.550670
----
 D)PA
wryvcj4u/AfoKBKyX3m.'6W/0 ,DT70E
u7kq5HN*iO'r 
----
iter 1000, loss: 85.405387
----
  wapthe, ase is gid hemeesy the.  vide rheyke winl 
----
iter 2000, loss: 68.977607
----
 lthe the lol, ,oohere jetharrl he tn"kas ron -he s 
----
iter 3000, loss: 60.775373
----
 e shalge cinge merocithefred ik. crifon, thy wad f 
----
iter 4000, loss: 56.615702
----
  hes ordyr Voryor si!etore, int ohpeg the ulr ivs  
----
iter 5000, loss: 59.539478
----
 wing cnithid onoly; the th pasp; withit this ouler 
----
iter 6000, loss: 60.948190
----
 y vouldont lell ave fe, sawas to bo nt ther an es  
----
iter 7000, loss: 56.350287
----
 n was his tay taknwis he the ha tentrun: moshatft  
----
iter 8000, loss: 52.939898
----
 mly d's capsist famey ntout ecurf tos cohe at has  
----
iter 9000, loss: 51.172342
----
  thouadioned the and tha fooln mum has -tored than 
----
iter 10000, loss: 50.422570
----
 hed wir ther ho doniched ristut the wastowet if wi 
----


In [10]:
sample(hprev, inputs[0], 5000)

----
 ing.

Tren the difing sackley anlenes erely., com? hid on wed wer took is sto hee horented thas hise noor. Mr gooe cale ald in ham, the then pmeas - cefrice Gregor uaning hak the in thawibldestas domeedent. Ote be-ik the in - ain souns frich siiger"

The asto has proucladivecone marky shad ap the wor toot him natt it where theikad if the deon of the susay sade whid at aad thell was therter, en picked mo had for hiy the reesit Dathding say team; and do the eser lasoede, lreanlm on ift hed dregozst ald wite pust wangthe cofh ther to shoved cher in woughad notlitt sad hit the wine dor moo kile do port leme wouved the moat to his whipn disiln han's it was, "o hared stemald aly toe nongl hes wand daints thete suiclease ving meean's wolw exwibellernwo the - theaf thas voows. dimtels, the waveamow; Gmegot didinge If? I cur the to theb ink and ove preitgut wory the hemr bome as eftat lene the loed the wey htars steven, hand gonkl oney eis thor, an legubln. Bud the one ""or frhaw onisty w